In [3]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from plotly.subplots import make_subplots
import statsmodels.api as sm # get data
profile = pd.read_csv('cps-school-info22.csv')
safety = pd.read_csv('cps_safety.csv')
crime = pd.read_csv('crimes-22.csv')
# merge profile & safety
merge = profile.merge(safety, on='School_ID',how='left') # drop columns 
clean = merge.drop(columns=[ 'Long_Name_y', 'Primary_Category_y', 'Zip_y', 'Student_Count_Total_y', 'Student_Count_Low_Income_y', 'Student_Count_Special_Ed_y', 'Student_Count_Black_y', 'Student_Count_Hispanic_y', 'Student_Count_White_y', 'Student_Count_Asian_y', 'Student_Count_Native_American_y', 'Student_Count_Asian_Pacific_Islander_y', 'Student_Count_Hawaiian_Pacific_Islander_y', 'Statistics_Description_y', 'Demographic_Description_y', 'Overall_Rating_y', 'School_Latitude_y', 'School_Longitude_y'])# df with necessary crime columns, create # of crimes column
crime_select = crime[[ 'Ward', 'Year']]
crime_select = crime_select[crime_select['Year'] == 2022]
crime_select = crime_select.groupby(['Ward', 'Year']).size().reset_index(name = 'num_crimes') # merge 3 dfs
merged_data = pd.merge(clean, crime_select, on='Ward', how='outer')

In [13]:
import pandas as pd
from matplotlib import pyplot
import plotly.express as px

import pymssql

# config file with credentials
from config import database
from config import username
from config import password
from config import server
from config import table

In [4]:
merged_data.to_csv('clara.csv')

In [5]:
dl_2022 = pd.read_csv('dl_2022.csv')

In [7]:
DLML1322 = dl_2022.loc[(dl_2022['Time Period1'] == 'EOY')]

DLMLF1322 = DLML1322[
    [
        'Category',
        'School Year',
        '# of Misconducts2',
        '# of Group 1-2 (minor) Misconducts3',
        '# of Group 3-4 (moderate) Misconducts4',
        '# of Group 5-6 (major) Misconducts5',
        '# of Police Notifications',
        '% of Misconducts Resulting in Police Notification'
    ]
]

DLMLF1322 = DLMLF1322.loc[
    ((DLMLF1322['Category'] == 'Grade PK-2') 
    | (DLMLF1322['Category'] == 'Grade 3-5') 
    | (DLMLF1322['Category'] == 'Grade 6-8') 
    | (DLMLF1322['Category'] == 'Grade 9-12'))
    ]

DLMLF1322['School Year'] = DLMLF1322['School Year'].astype('category').cat.codes

In [8]:
DLMLF1322.to_csv('correlation.csv')

In [10]:
wards = pd.read_csv('wards.csv')

school_comm = wards[['School ID', 'Community Area Number']]
sl_2022 = pd.read_csv('sl_2022.csv')

In [11]:
misconduct = sl_2022[
    [
        'School ID', 
        '# of Misconducts', 
        '# of Suspensions (includes ISS and OSS)',
        '# of Students Expelled',
        '# of Police Notifications'
        ]
    ]

misconduct = misconduct[misconduct['# of Misconducts'] != '--']
misconduct = misconduct[misconduct['# of Suspensions (includes ISS and OSS)'] != '--']
misconduct = misconduct[misconduct['# of Police Notifications'].astype('int64') != '--']

In [14]:
query = f'''
SELECT 
    community_area,
    count(community_area) AS num_crimes
FROM {table}
WHERE community_area IS NOT NULL
GROUP BY community_area
ORDER BY community_area
'''

con = pymssql.connect(server, username, password, database)

comm = pd.read_sql(query, con)

# Cleaning / Organizing
comm['community_area'] = comm['community_area'].astype('int')
comm = comm.sort_values('community_area').reset_index(drop=True)

c:\Users\JassleenBhullar\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [15]:
school_mis = misconduct.merge(
    school_comm,
    how = 'inner',
    on = 'School ID',
    )

school_df = school_mis.merge(
    comm,
    how= 'inner',
    left_on = 'Community Area Number',
    right_on = 'community_area' 
).drop(columns='Community Area Number')

# Cleaning
school_df['# of Misconducts'] = school_df['# of Misconducts'].astype('int')
school_df['# of Suspensions (includes ISS and OSS)'] = school_df['# of Suspensions (includes ISS and OSS)'].astype('int')
school_df['# of Police Notifications'] = school_df['# of Police Notifications'].astype('int')

In [16]:
miscon_crimes = school_df.groupby(['community_area']) \
.agg(
    {
    '# of Misconducts': 'sum',
    '# of Suspensions (includes ISS and OSS)' : 'sum',
    '# of Students Expelled': 'sum',
    '# of Police Notifications' : 'sum',
    'num_crimes' : 'mean',
    }
)

In [18]:
miscon_crimes.to_csv('miscon_crimes.csv')